#Laboratorio 10: uso de SageMaker Pipelines y SageMaker Model Registry con SageMaker Studio

En este laboratorio, usted crea y ejecuta una canalización de Amazon SageMaker y supervisa su progreso. También ubica y explora algunos de los artefactos que el proceso de machine learning (ML) utiliza o genera.

Si el tiempo lo permite, también puede revisar los detalles de linaje del modelo que generó la canalización.

## Tarea 2.1: configuración del entorno

Antes de crear su canalización de SageMaker, debe instalar los paquetes necesarios, importar módulos y organizar los archivos de apoyo para preparar el entorno. Esta canalización se diseñó para utilizar un grupo de funciones, para que también pueda crear un grupo de funciones en Amazon SageMaker Feature Store y ejecutar un flujo de Data Wrangler para preparar su entorno. 

Ejecute las celdas de esta tarea para hacer lo siguiente:
- Instalar dependencias.
- Importar los módulos necesarios.
- Copiar datos y código en Amazon Simple Storage Service (Amazon S3).
- Crear un grupo de funciones.
- Ingerir funciones en el grupo de funciones.

### Tarea 2.1.1: instalar dependencias

In [ ]:
#install dependencies
%pip install --upgrade pip 
%pip install pytest-astropy ==  0.7.0
%pip install rsa == 4.7.2
%pip install PyYAML
!apt update && apt install -y git
%pip install git+https://github.com/aws-samples/ml-lineage-helper

### Tarea 2.1.2: importar módulos

In [ ]:
#import-modules
import os
import json
import boto3
import sagemaker
import sagemaker_datawrangler
import sagemaker.session
import datetime as dt
import pandas as pd
import time
from time import gmtime, strftime
import uuid
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.transformer import Transformer
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from ml_lineage_helper import *
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

In [ ]:
#create sessions
boto_session  =  boto3.Session()
sagemaker_session = sagemaker.Session()


In [ ]:
#create clients
s3_client = boto3.client('s3')
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')
sagemaker_client = boto3.client('sagemaker')

In [ ]:
#feature store session
feature_store_session = Session(
    boto_session = boto_session,
    sagemaker_client = sagemaker_client,
    sagemaker_featurestore_runtime_client = featurestore_runtime
)

In [ ]:
#set global variables
default_bucket = sagemaker_session.default_bucket()
region = boto_session.region_name
role = sagemaker.get_execution_role()

### Tarea 2.1.3: copiar archivos del laboratorio en Amazon S3 

In [ ]:
# Upload files to default bucket
s3_client.put_object(Bucket = default_bucket, Key = 'data/')
s3_client.put_object(Bucket = default_bucket, Key = 'input/code/')
s3_client.upload_file('pipelines/data/storedata_total.csv', default_bucket, 'data/storedata_total.csv')
s3_client.upload_file('pipelines/input/code/evaluate.py', default_bucket, 'input/code/evaluate.py')
s3_client.upload_file('pipelines/input/code/generate_config.py', default_bucket, 'input/code/generate_config.py')
s3_client.upload_file('pipelines/input/code/processfeaturestore.py', default_bucket, 'input/code/processfeaturestore.py')

# Preview the dataset
print('Dataset preview:')
customer_data = pd.read_csv('pipelines/data/storedata_total.csv')
customer_data.head()

### Tarea 2.1.4: crear el grupo de funciones

En esta tarea, cree un grupo de funciones para los datos. Primero, cree un esquema de los datos. En este laboratorio, el esquema debe ordenarse primero por la columna **name** (nombre), y después por la variable **type** (tipo).

In [ ]:
#set-up-feature-store-variables
record_identifier_feature_name = 'FS_ID'
event_time_feature_name = 'FS_time'

column_schemas = [
    {
        "name": "retained",
        "type": "long"
    },
    {
        "name": "esent",
        "type": "long"
    },
    {
        "name": "eopenrate",
        "type": "float"
    },
    {
        "name": "eclickrate",
        "type": "float"
    },
    {
        "name": "avgorder",
        "type": "float"
    },
    {
        "name": "ordfreq",
        "type": "float"
    },
    {
        "name": "paperless",
        "type": "long"
    },
    {
        "name": "refill",
        "type": "long"
    },
    {
        "name": "doorstep",
        "type": "long"
    },
    {
        "name": "first_last_days_diff",
        "type": "long"
    },
    {
        "name": "created_first_days_diff",
        "type": "long"
    },
    {
        "name": "favday_Friday",
        "type": "long"
    },
    {
        "name": "favday_Monday",
        "type": "long"
    },
    {
        "name": "favday_Saturday",
        "type": "long"
    },
    {
        "name": "favday_Sunday",
        "type": "long"
    },
    {
        "name": "favday_Thursday",
        "type": "long"
    },
    {
        "name": "favday_Tuesday",
        "type": "long"
    },
    {
        "name": "favday_Wednesday",
        "type": "long"
    },
    {
        "name": "city_BLR",
        "type": "long"
    },
    {
        "name": "city_BOM",
        "type": "long"
    },
    {
        "name": "city_DEL",
        "type": "long"
    },
    {
        "name": "city_MAA",
        "type": "long"
    },
    {
        "name": "FS_ID",
        "type": "long"
    },
    {
        "name": "FS_time",
        "type": "float"
    }
]


Ahora, cree el grupo de funciones

In [ ]:
# Flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = 'featureengineer'
flow_export_id = f"{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

# Feature group name, with flow_name and a unique id. You can give it a customized name
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"

# SageMaker Feature Store writes the data in the offline store of a Feature Group to a 
# Amazon S3 location owned by you.
feature_store_offline_s3_uri = 's3://' + default_bucket

# Controls if online store is enabled. Enabling the online store allows quick access to 
# the latest value for a record by using the GetRecord API.
enable_online_store = True

In [ ]:
#create-feature-group
default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name = column_schema['name'], 
        feature_type = column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]


print(f"Feature Group Name: {feature_group_name}")

# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup = 'primary',
        ConfigurationUpdates = {
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

feature_group = FeatureGroup(
    name = feature_group_name, sagemaker_session = feature_store_session, feature_definitions = feature_definitions)

feature_group.create(
    s3_uri = feature_store_offline_s3_uri,
    record_identifier_name = record_identifier_feature_name,
    event_time_feature_name = event_time_feature_name,
    role_arn = role,
    enable_online_store = enable_online_store
)

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"Feature Group {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group = feature_group)


### Tarea 2.1.5: ingerir funciones

Este proceso tarda aproximadamente 8 minutos en completarse.

In [ ]:
#populate-feature-store
column_list = ['retained','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday', 'favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA','FS_ID','FS_time']
lab_test_data = pd.read_csv('featureengineer_data/store_data_processed.csv', names = (column_list), header = 1)
feature_group.ingest(data_frame = lab_test_data, wait = True)

### Tarea 2.2: crear y ejecutar una canalización de SageMaker

Ahora que su entorno está configurado, configure, cree e inicie una canalización de SageMaker. 

Una canalización de SageMaker es un flujo de trabajo que ejecuta una serie de pasos dependientes. Los pasos pueden aceptar entradas y enviar salidas, para que los datos y otros activos puedan pasar entre ellos. 

Ejecute las siguientes celdas para hacer lo siguiente:
- Definir las variables que se necesitan para configurar la canalización.
- Configurar una sesión de SageMaker.
- Definir los pasos de la canalización.
- Configurar la canalización.
- Crear la canalización.
- Iniciar la canalización.
- Describir la canalización.
- Crear un evento de espera para que el cuaderno no proceda hasta que la canalización haya terminado de ejecutarse.

### Tarea 2.2.1: configurar las variables que utiliza la canalización

In [ ]:

#pipeline-variables
feature_group_name = feature_group.name
model_name = "Churn-model"

sklearn_processor_version = "0.23-1"
model_package_group_name = "ChurnModelPackageGroup"
pipeline_name = "ChurnModelSMPipeline"

processing_instance_count = ParameterInteger(
    name = "ProcessingInstanceCount",
    default_value = 1
    )

processing_instance_type = ParameterString(
        name = "ProcessingInstanceType",
        default_value = "ml.m5.xlarge"
    )

training_instance_type = ParameterString(
        name = "TrainingInstanceType",
        default_value = "ml.m5.xlarge"
    )

input_data = ParameterString(
        name = "InputData",
        default_value = "s3://{}/data/storedata_total.csv".format(default_bucket), 
    )

batch_data = ParameterString(
        name = "BatchData",
        default_value = "s3://{}/data/batch/batch.csv".format(default_bucket),
    )

### Tarea 2.2.2: configurar la canalización

Usted define una canalización llamada **ChurnModelPipeline** para producir un modelo que evalúe la posibilidad de conservar o perder clientes. Esta canalización tienen nueve pasos. 

Cada paso en una canalización ejecuta un tipo de trabajo específico. Las entradas requeridas para un trabajo varían en función del tipo de trabajo. Consulte [Tipos de pasos](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#build-and-manage-steps-types) para obtener más información sobre los tipos de pasos de la canalización de SageMaker.

Revise el código en las siguientes celdas para comprender cada paso que se define:

El paso **ChurnModelProcess** se define en la variable llamada **step_process**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Processing (Procesamiento): los trabajos de procesamiento se definen con la clase ProcessingStep().
- **Processor** (Procesador): SKLearnProcessor.
- **Destination** (Destino): la salida se enviará a las carpetas que defina en el bucket de S3 predeterminado.
- **Job Arguments** (Argumentos del trabajo): este paso usará el almacén de funciones para procesar el conjunto de datos.
- **Code** (Código): **processfeaturestore.py**, que reside en su bucket de S3 predeterminado.


In [ ]:
#configure-processing-step
# Run a scikit-learn script to do data processing on SageMaker 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version = sklearn_processor_version,
        instance_type = processing_instance_type.default_value, 
        instance_count = processing_instance_count,
        sagemaker_session = sagemaker_session,
        role = role,
    )

# Inputs, outputs, and code are parameters to the processor
# step_* will become the pipeline steps toward the end of the cell
# in this case, use the feature store as input, so there is no externalinput
step_process = ProcessingStep(
        name = "ChurnModelProcess",
        processor = sklearn_processor,
        outputs = [
            ProcessingOutput(output_name = "train", source = "/opt/ml/processing/train",\
                             destination = f"s3://{default_bucket}/output/train" ),
            ProcessingOutput(output_name = "validation", source = "/opt/ml/processing/validation",\
                            destination = f"s3://{default_bucket}/output/validation"),
            ProcessingOutput(output_name = "test", source = "/opt/ml/processing/test",\
                            destination = f"s3://{default_bucket}/output/test"),
            ProcessingOutput(output_name = "batch", source = "/opt/ml/processing/batch",\
                            destination = f"s3://{default_bucket}/data/batch"),
            ProcessingOutput(output_name = "baseline", source = "/opt/ml/processing/baseline",\
                            destination = f"s3://{default_bucket}/input/baseline")
        ],
        job_arguments = ["--featuregroupname",feature_group_name,"--default-bucket",default_bucket,"--region",region],
        code = f"s3://{default_bucket}/input/code/processfeaturestore.py",
    )

El paso **ChurnHyperParameterTuning** se define en la variable llamada **step_tuning**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Tuning (Ajuste): los trabajos de ajuste se definen con la clase TuningStep().
- **Tuner** (Ajustador): este trabajo utiliza el marco de trabajo de XGBoost.
- **Inputs** (Entradas): observe que este trabajo utiliza los datos de entrenamiento y validación que obtenidos de paso ChurnModelProcess, **step_process**.

In [ ]:
#configure-churn-hyperparameter-tuning
# Training/tuning step for generating model artifacts
model_path = f"s3://{default_bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework = "xgboost",
    region = region,
    version = "1.5-1",
    py_version = "py3",
    instance_type = training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
    }

xgb_train = Estimator(
    image_uri = image_uri,
    instance_type = training_instance_type,
    instance_count = 1,
    hyperparameters = fixed_hyperparameters,
    output_path = model_path,
    base_job_name = f"churn-train",
    sagemaker_session = sagemaker_session,
    role = role
    )

In [ ]:
#Tuning steps
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
    }
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "ChurnHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs = 2, max_parallel_jobs = 2),
    inputs = {
            "train": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
            "validation": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
        },
    )

El paso **ChurnEvalBestModel** se define en la variable llamada **step_eval**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Processing (Procesamiento).
- **Processor** (Procesador): ScriptProcessor.
- **Inputs** (Entradas): observe que este trabajo utiliza el modele superior de ChurnHyperParameterTuning (**step_tuning**) y la salida de prueba de ChurnModelProcess (**step_process**).
- **Outputs** (Salidas): la salida se escribe en el bucket de S3 predeterminado.
- **Code** (Código): un script llamado **evaluate.py**, que reside en Amazon S3, se usa para la evaluación.

In [ ]:
#configure-churn-best-model
evaluation_report = PropertyFile(
    name = "ChurnEvaluationReport",
    output_name = "evaluation",
    path = "evaluation.json",
)

script_eval = ScriptProcessor(
    image_uri = image_uri,
    command = ["python3"],
    instance_type = processing_instance_type,
    instance_count = 1,
    base_job_name = "script-churn-eval",
    role = role,
    sagemaker_session = sagemaker_session,
)

step_eval = ProcessingStep(
    name = "ChurnEvalBestModel",
    processor = script_eval,
    inputs = [
        ProcessingInput(
            source = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
            destination = "/opt/ml/processing/model"
        ),
        ProcessingInput(
            source = step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination = "/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(output_name = "evaluation", source = "/opt/ml/processing/evaluation",\
                            destination = f"s3://{default_bucket}/output/evaluation"),
    ],
    code = f"s3://{default_bucket}/input/code/evaluate.py",
    property_files = [evaluation_report],
)

El paso **ChurnCreateModel** se define en la variable llamada **step_create_model**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Model (Modelo): los trabajos de modelo se definen con la clase Model().
- **Model** (Modelo): el modelo que usa este paso se define en la variable que se definió previamente llamada **model**. Observe que la variable **model** utiliza el modelo superior que se creó con ChurnHyperParameterTuning (**step_tuning**).
- **Inputs** (Entradas): las entradas incluyen un tipo de instancia y un tipo de acelerador.

In [ ]:
#configure-model-creation
model = Model(
    image_uri = image_uri,        
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0,s3_bucket = default_bucket,prefix = "output"),
    name = model_name,
    sagemaker_session = sagemaker_session,
    role = role,
)

inputs = CreateModelInput(
    instance_type = "ml.m5.large",
    accelerator_type = "ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name = "ChurnCreateModel",
    model = model,
    inputs = inputs,
)

El paso **ChurnModelConfigFile** se define en la variable llamada **step_config_file**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Processing (Procesamiento).
- **Processor** (Procesador): ScriptProcessor.
- **Code** (Código): **generate_config.py**, que reside en su bucket de S3 predeterminado.
- **Job Arguments** (Argumentos del trabajo): los argumentos del trabajo incluyen al modelo que se generó con **ChurnCreateModel**, la ruta al informe de sesgos, el bucket predeterminado, el número de muestras y el número de instancias que se usan para el procesamiento.
- **Depends On** (Depende de): observe que este trabajo no se puede ejecutar hasta que la creación del modelo no haya finalizado.

In [ ]:

#configure-script-processing
bias_report_output_path = f"s3://{default_bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
analysis_config_path = f"s3://{default_bucket}/clarify-output/bias/analysis_config.json"
clarify_image = sagemaker.image_uris.retrieve(framework = 'sklearn', version = sklearn_processor_version, region = region)

#custom_image_uri = None
script_processor = ScriptProcessor(
    command = ['python3'],
    image_uri = clarify_image,
    role = role,
    instance_count = 1,
    instance_type = processing_instance_type,
    sagemaker_session = sagemaker_session,
)

step_config_file = ProcessingStep(
    name = "ChurnModelConfigFile",
    processor = script_processor,
    code = f"s3://{default_bucket}/input/code/generate_config.py",
    job_arguments = ["--modelname", step_create_model.properties.ModelName, "--bias-report-output-path", bias_report_output_path, "--clarify-instance-type", clarify_instance_type,\
                  "--default-bucket", default_bucket, "--num-baseline-samples", "50", "--instance-count", "1"],
    depends_on = [step_create_model.name]
)

El paso **ChurnTransform** se define en la variable llamada **step_transform**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Transform (Transformación): los trabajos de transformación se definen con la clase TransformStep().
- **Transformer** (Transformador): los detalles del transformador se configuran en la variable definida previamente llamada **transformer**. Observe que esta variable utiliza el modelo que se creó en ChurnCreateModel (**step_create_model**).
- **Inputs** (Entradas): los datos que se transformarán, batch.csv, se definieron en el cuaderno anterior. La entrada incluye el tipo de archivo y cómo se debe dividir.

In [ ]:
#configure-inference
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type = "ml.m5.xlarge",
    instance_count = 1,
    assemble_with = "Line",
    accept = "text/csv",    
    output_path = f"s3://{default_bucket}/ChurnTransform"
    )

step_transform = TransformStep(
    name = "ChurnTransform",
    transformer = transformer,
    inputs = TransformInput(data = batch_data, content_type = "text/csv", join_source = "Input", split_type = "Line")
    )

El paso **ClarifyProcessingStep** se define en la variable llamada **step_clarify**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Processing (Procesamiento).
- **Processor** (Procesador): este trabajo utiliza SageMakerClarifyProcessor. Puede revisar la configuración del procesador en la variable llamada **clarify_processor**.
- **Inputs** (Entradas): las entradas se definen en las variables **data_input** y **congif_input**.
- **Outputs** (Salidas): la salida se escribe en una carpeta dentro del bucket predeterminado. 
- **Depends On** (Depende de): observe que este trabajo no se puede ejecutar hasta que **ChurnModelConfigFile** no haya creado la configuración del archivo que requiere Amazon SageMaker Clarify. 

In [ ]:
#configure-clarify-processing
data_config = sagemaker.clarify.DataConfig(
s3_data_input_path = f's3://{default_bucket}/output/train/train.csv',
s3_output_path = bias_report_output_path,
    label = 0,
    headers = ['target','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday','favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA'],
    dataset_type = "text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role = role,
    instance_count = 1,
    instance_type = clarify_instance_type,
    sagemaker_session = sagemaker_session,
)

config_input = ProcessingInput(
    input_name = "analysis_config",
    source=analysis_config_path,
    destination = "/opt/ml/processing/input/analysis_config",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_compression_type = "None",
    )

data_input = ProcessingInput(
    input_name = "dataset",
    source = data_config.s3_data_input_path,
    destination = "/opt/ml/processing/input/data",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_data_distribution_type = data_config.s3_data_distribution_type,
    s3_compression_type = data_config.s3_compression_type,
)

result_output = ProcessingOutput( 
    source = "/opt/ml/processing/output",
    destination = data_config.s3_output_path,
    output_name = "analysis_result",
    s3_upload_mode = "EndOfJob",
)

step_clarify = ProcessingStep(
    name = "ClarifyProcessingStep",
    processor = clarify_processor,
    inputs = [data_input, config_input],
    outputs = [result_output],
    depends_on = [step_config_file.name]
)

El paso **RegisterChurnModel** se define en la variable llamada **step_register**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Register Model (Registro del modelo): los trabajos de registro se definen con la clase RegisterMode().
- **Estimator** (estimador): el estimador se define antes en la celda en la variable **xgbtrain**.
- **Model Data** (Datos del modelo): este es el URI del modelo que se genera en **ChurnHyperParameterTuning**.
- **Content Types** (Tipos de contenido): text/csv
- **Response Types** (Tipos de respuestas): text/csv
- **Inference Instance** (Instancia de inferencia): es el tipo de instancia que se utilizará para el proceso de inferencia.
- **Transform Instance** (Instancia de transformación): es el tipo de instancia que se utilizará para el proceso de transformaciones.
- **Model Package Group Name** (Nombre del grupo de paquetes de modelos): este es el nombre del grupo que almacenará el grupo de versiones de modelos.
- **Model Metrics** (Métricas del modelo): esto define la ubicación de las métricas del modelo. Los archivos que se incluyen son el informe de sesgos y el informe de explicabilidad de SageMaker Clarify y la evaluación del modelo. 

In [ ]:
#configure-model-registry
model_statistics = MetricsSource(
    s3_uri = "s3://{}/output/evaluation/evaluation.json".format(default_bucket),
    content_type = "application/json"
    )
explainability = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    )

bias = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    ) 

model_metrics = ModelMetrics(
    model_statistics = model_statistics,
    explainability = explainability,
    bias = bias
)

step_register = RegisterModel(
    name = "RegisterChurnModel",
    estimator = xgb_train,
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
    content_types = ["text/csv"],
    response_types = ["text/csv"],
    inference_instances = ["ml.t2.medium", "ml.m5.large"],
    transform_instances = ["ml.m5.large"],
    model_package_group_name = model_package_group_name,
    model_metrics = model_metrics,
)

El paso **CheckAUCScoreChurnEvaluation** se define en la variable llamada **step_cond**. 

La configuración del paso incluye lo siguiente:
- **Type** (Tipo) Condition (Condición): los trabajos de condición se definen con la clase ConditionStep().
- **Conditions** (Condiciones): esta condición se evalúa como True (Verdadera) si la salida de **ChurnEvalBestModel** es mayor a 0,75.
- **If Steps** (Pasos si): esta es la lista de pasos que se ejecutan si la condición se evalúa como True (Verdadera).
- **If Steps** (Pasos si no): esta es la lista de pasos que se ejecutan si la condición se evalúa como False (Falsa). Observe que esta lista está en blanco, lo que significa que la canalización detiene el procesamiento si la condición no se cumple.

In [ ]:
%%capture
cond_lte = ConditionGreaterThan(
    left = JsonGet(
        step = step_eval,
        property_file = evaluation_report,
        json_path = "binary_classification_metrics.auc.value"
    ),
    right = 0.75,
)

step_cond = ConditionStep(
    name = "CheckAUCScoreChurnEvaluation",
    conditions = [cond_lte],
    if_steps = [step_create_model, step_config_file, step_transform, step_clarify, step_register],
    else_steps = [],
)

### Tarea 2.2.3: definir la canalización

Después de definir los pasos, configure la canalización en la variable llamada **pipeline**. Observe cómo los pasos que se definieron previamente pasan a la definición de la canalización.

In [ ]:
 #define pipeline function
def get_pipeline(
    region,
    role = None,
    default_bucket = None,
    model_package_group_name = "ChurnModelPackageGroup",
    pipeline_name = "ChurnModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version = None
    ):

    #configure pipeline instance
    pipeline = Pipeline(
        name = pipeline_name,
        parameters = [
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps = [step_process, step_tuning, step_eval, step_cond],
        sagemaker_session = sagemaker_session
    )
    return pipeline


### Tarea 2.2.4: crear la canalización

In [ ]:
 #create pipeline using function
pipeline = get_pipeline(
  region = region,
    role = role,
    default_bucket = default_bucket,
    model_package_group_name = model_package_group_name,
    pipeline_name = pipeline_name,
    custom_image_uri = clarify_image,
    sklearn_processor_version = sklearn_processor_version
)

### Tarea 2.2.5: actualizar la canalización para utilizar el rol de IAM correcto. 

In [ ]:
#set-iam-role
pipeline.upsert(role_arn = role)

**Nota:** Si se encuentra con la siguiente advertencia después de ejecutar la celda, puede ignorarla sin problemas.

“No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config”. (“No se ha encontrado ningún trabajo de entrenamiento finalizado asociado a este estimador. Por favor, asegúrese de que este estimador solo se utiliza para crear la configuración del flujo de trabajo”).

### Tarea 2.2.6: iniciar la canalización

In [ ]:
#start-pipeline
RunPipeline = pipeline.start()

### Tarea 2.2.7: describir la canalización

In [ ]:
#describe-pipeline
RunPipeline.describe()

La canalización demora unos 35 minutos para ejecutarse.

Mientras se ejecuta la canalización, continúe con la siguiente tarea para explorar la canalización en la consola de Amazon SageMaker Studio.

## Tarea 2.3: supervisar y aprobar la canalización

En esta tarea, explore la canalización con la consola de Amazon SageMaker Studio.

### Tarea 2.3.1: supervisar la canalización en SageMaker Studio

La próxima tarea abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_10.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **lab_10.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora, puede ver las instrucciones mientras explora los pasos de la canalización.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando haya terminado de explorar los pasos de la canalización, vuelva al cuaderno. Para ello, seleccione la pestaña **lab_10.ipynb**.

1. En SageMaker Studio, seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
1. Seleccione **Pipelines** (Canalizaciones).

Se abre la pestaña **Pipelines** (Canalizaciones) en SageMaker Studio.

1. Seleccione la canalización llamada **ChurnModelSMPipeline**. 

SageMaker Studio abre la pestaña **ChurnModelSMPipeline**.

1. En la pestaña **ChurnModelSMPipeline**, en **Executions** (Ejecuciones), abra (haciendo clic con el botón derecho del mouse) el estado de la canalización y luego seleccione **Open execution details** (Abrir los detalles de la ejecución). 

SageMaker Studio abre la página **Directed Acyclic Graph** (DAG).

Directed Acyclic Graph (DAG) muestra el flujo del trabajo y el proceso de la canalización. Los colores se utilizan para indicar el estado de un paso. Los colores indicadores son los siguientes:
- **gris**: en espera de ejecución.
- **azul**: en ejecución.
- **verde**: se completó con éxito.
- **rojo**: error.

Cada paso que explora tiene las mismas cuatro pestañas en el panel de detalles del paso. Aunque los títulos de las pestañas son iguales, los contenidos de cada pestaña varían en función del tipo de trabajo y de su configuración:

- **Input** (Entrada): esta pestaña contiene entadas que se han pasado al trabajo. Los ejemplos de entradas son los parámetros que especifican los tipos de instancias, los roles de AWS Identity and Access Management (IAM) o los argumentos necesarios para ejecutar el trabajo. Otros ejemplos de entradas incluyen archivos como código, conjuntos de datos e imágenes de Docker.
- **Output** (Salida): esta pestaña muestra la salida que creó el trabajo. Algunos ejemplos de salidas son métricas, gráficos, archivos y resultados de evaluaciones.
- **Logs** (Registros): esta pestaña proporciona una lista de registros a asociados con el trabajo. Si el usuario cuenta con suficientes privilegios para los Registros de Amazon CloudWatch, puede seleccionar el vínculo de los registros y ver los mensajes de registro detallados en CloudWatch. Algunos tipos de trabajo no generan registros.
- **Information** (Información): esta pestaña proporciona información básica sobre un trabajo, como el tipo, el nombre y el momento en que se ejecutó.

1. Seleccione el paso llamado **ChurnModelProcess**. Se abrirá un nuevo panel llamado **ChurnModelProcess**.
1. En el panel **ChurnModelProcess**, revise las pestañas asociadas con este paso de la canalización: 
    - Seleccione la pestaña **Input** (Entrada). Esta pestaña contiene información útil sobre los parámetros y archivos que se utilizan en el paso de procesamiento. En la lista de parámetros, están los detalles, incluido el tipo de instancia y la imagen que utiliza el trabajo, la ubicación del conjunto de datos, la ubicación del código y los destinos para las diferentes salidas que se generan. Deslícese hasta la parte inferior del panel para hallar los archivos de entrada que se pasaron al trabajo.
    - Seleccione la pestaña **Output** (Salida). En esta pestaña se muestran los diferentes archivos que se generan en el paso de la canalización y dónde están ubicados. Esta canalización ubica a todas las salidas en el bucket predeterminado de SageMaker Studio.
    - Seleccione la pestaña **Logs** (Registros). Esta pestaña muestra los registros que se generan en el trabajo. Tener los registros disponibles dentro de SageMaker Studio acelera la investigación y la resolución de problemas cuando un paso de la canalización no se ejecuta en forma correcta.
    - Seleccione la pestaña**Information** (Información). Esta pestaña proporciona información general de alto nivel sobre el paso de la canalización. Incluye información como el tipo y el nombre del paso y un vínculo al registro del trabajo. También proporciona detalles sobre cuándo se ejecutó el trabajo y cuánto tiempo duró.
        - Observe que el **Step Type** (Tipo de paso) es **Processing** (Procesamiento).

### Tarea 2.3.2: descubrir los detalles del paso de la canalización

En los siguientes pasos, seleccione el nodo apropiado del gráfico acíclico direccionado (DAG) para hallar información sobre un paso de canalización determinado. Si necesita ayuda para hallar las respuestas, las respuestas correctas o sugerencias se incluyen al final de este cuaderno de Python.

1. En el paso llamado **ChurnHyperParameterTuning**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Cuál fue el **Overall Best Training Job** (Mejor trabajo de entrenamiento en general) que se generó en este paso?
1. En el paso llamado **ChurnEvalBestModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Cuál es el nombre del script de Python que se utiliza para evaluar el modelo superior que se identificó en el paso anterior?
    - ¿Dónde se ubica el archivo?
    - ¿Dónde se escriben los resultados de este paso?
1. En el paso llamado **CheckAUCScoreChurnEvaluation**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Cuál fue el **Evaluation outcome** (Resultado de la evaluación)?
1. En el paso llamado **ChurnCreateModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Este trabajo generó algún registro?
1. En el paso llamado **RegisterChurnModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Cuál es el valor de la métrica del área bajo la curva (AUC)?
1. En el paso llamado **ChurnTransform**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    - ¿Este trabajo generó registros?
    - ¿Qué archivos fueron las entradas para este trabajo?
1. En el paso llamado **ChurnModelConfigFile**, ubique los siguientes detalles: 
    - ¿Qué ProcessingInstanceType (Tipo de instancia de procesamiento) se utilizó para ejecutar esta trabajo?
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
1. En el paso llamado **ClarifyProcessingStep**, ubique los siguientes detalles:
    - ¿Cuál fue el archivo de salida de este paso?
    - ¿Dónde se escribió la salida?

## Tarea 2.3.3: aprobar el modelo en la canalización

1. Una vez que la canalización haya finalizado la ejecución, revise el modelo que se creó en **Model registry** (Registro del modelo).
    - En SageMaker Studio, seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
    - Expanda la lista **Models** (Modelos).
    - Seleccione **Model registry** (Registro del modelo).
    - Abra el grupo del modelo llamado **ChurnModelPackageGroup**.
    - En la pestaña **ChurnModelPackageGroup**, abra (haciendo clic con el botón derecho del mouse) la fila en la tabla **Versions** (Versiones) y seleccione **Open model version** (Abrir modelo de versión). Observe que el estado del modelo es **Pending** (Pendiente). Además, observe que el valor de **Execution** (Ejecución) es el nombre de la canalización que se acaba de completar.

    Los detalles adicionales sobre la canalización se encuentran en cada una de las siguientes pestañas:
    - **Activity** (Actividad): en esta pestaña se muestra la actividad del modelo. Contiene información del evento y cuánto tiempo ha transcurrido desde la última modificación del modelo.
    - **Model quality** (Calidad del modelo): en esta pestaña se muestra las métricas de exactitud del modelo.
    - **Explainability** (Explicabilidad): en esta pestaña se muestra la importancia de las funciones del modelo en términos de valores de Shapley (SHAP).
    - **Bias report** (Informe de sesgos): en esta pestaña se muestran los posibles sesgos del modelo.
    - **Inference recommender** (Recomendador de inferencias): en esta pestaña se proporcionan recomendaciones para mejorar el rendimiento de precios de un modelo. Esta pestaña no contiene datos porque esta función no es compatible con este modelo de paquete.
    - **Load test** (Prueba de carga): desde esta pestaña puede iniciar pruebas para evaluar diferentes tipos de instancias y evaluarlas para las métricas de rendimiento y latencia requeridas para una implementación de producción.
    - **Settings** (Configuración): en esta pestaña se muestra información como cuándo se creó el modelo, qué canalización lo generó, dónde se encuentra ubicado y el componente de prueba asociado a él.

1. Apruebe el modelo. Este proceso está diseñado para una revisión manual antes de aprobar el modelo. Sin embargo, es posible automatizar la aprobación del modelo dentro de la canalización.
    - Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span> (Actualizar estado).
    - Abra la lista del menú desplegable y seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">**Approved**</span> (Aprobado).
    - Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span> (Actualizar estado).
1. Cierre la pestaña **ChurnModelPackageGroup**.

### Tarea 2.3.4: ver los pasos de la canalización con el AWS SDK

Además de utilizar la interfaz de usuario de SageMaker Studio para ver los detalles de la canalización, puede utilizar los comandos del AWS SDK. Por ejemplo, el siguiente comando devuelve una lista de los pasos de la canalización.

In [ ]:
#list-steps
RunPipeline.list_steps()

## Tarea 2.4: revisar los artefactos

La próxima tarea abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_10.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **lab_10.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora, puede ver las instrucciones mientras explora los artefactos.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar los artefactos, seleccione la pestaña **lab_10.ipynb** para volver al cuaderno.

### Tarea 2.4.1: revisar los artefactos en SageMaker Studio

Mientras se ejecuta la canalización, cada paso generó artefactos como archivos, parámetros de entrenamiento y modelos. Puede identificar los artefactos que creó la canalización en SageMaker Studio.
1. Regrese a la pestaña llamada **ChurnModelSMPipeline**.
1. Seleccione la pestaña **Executions** (Ejecuciones).
1. Abra (haciendo clic con el botón derecho del mouse) la ejecución que figura en la lista y seleccione **View trial components generated by execution** (Ver componentes de la prueba generados en la ejecución). 

SageMaker Studio abre una nueva pestaña llamada **Trial Component List**. 

Se muestra una lista de todos los trabajos que ejecutó la canalización.

Observe que cada componente de la prueba tiene un **Trial Component Type** (Tipo de componente de la prueba). La información disponible en las distintas pestañas de los detalles de la prueba asociada dependes del tipo de componente de la prueba. No todas las pestañas de los detalles de la prueba contienen datos para todos los tipos de componentes.

1. Abra (haciendo clic con el botón derecho del mouse) la primera fila de la lista de trabajos y seleccione **Open in trial details** (Abrir en detalles de la prueba). 

SageMaker Studio abre una nueva pestaña llamada **Describe Trial Component** (Describir el componente de la prueba). 

En **Trial Components** (Componentes de la prueba), hay muchas pestañas disponibles. Dependiendo de lo que estaba haciendo el paso de la canalización, algunas pestañas podrían estar vacías. 

1. Seleccione la pestaña **Artifacts** (Artefactos). Los detalles de la entrada y la salida se utilizaron en el paso.
1. Seleccione la pestaña **Explainability** (Explicabilidad). En esta pestaña se muestra el informe de explicabilidad que generó SageMaker Clarify.
1. Seleccione la pestaña **Bias Report** (Informe de sesgos). En esta pestaña se muestra el informe de sesgos que generó SageMaker Clarify.

### Tarea 2.4.2: localizar los artefactos en el bucket de S3 predeterminado
**Nota:** use la Consola de administración de AWS para esta tarea. Una vez que haya explorado el bucket de S3, regrese a la pestaña del navegador donde está abierto SageMaker Studio y seleccione la pestaña **lab_10.ipynb**.

1. En la pestaña del navegador donde se encuentra abierta la consola, navegue hasta Amazon S3.
1. Seleccione el nombre del bucket que comience con **sagemaker-** y la región de AWS; por ejemplo, **sagemaker-us-west-2-123456789**.
1. Explore las carpetas y archivos de este bucket. Este bucket contiene el conjunto de datos, las entradas y salidas del procesamiento, los resultados de SageMaker Clarify y otros archivos que contribuyen al modelo resultante.
1. Regrese a la pestaña del navegador donde está abierto SageMaker Studio y seleccione la pestaña **lab_10.ipynb**.

## Tarea 2.5 (Opcional): crear y revisar el linaje de la canalización

Aprendió cómo usar SageMaker Clarify para explicar cómo un modelo hace predicciones y comprender los posibles sesgos de un modelo. También puede usar SageMaker Clarify para descubrir los pasos que se usan para generar el modelo, que suelen ser necesarios para la auditoría del modelo. En esta tarea, se aprovecha del módulo MLLineageHelper para crear el linaje de la ejecución actual de la canalización. Consulte [MLLineageHelper](https://github.com/aws-samples/ml-lineage-helper) para obtener más información sobre ML Lineage Helper.

El seguimiento del linaje de ML de Amazon SageMaker crea y almacena información sobre los pasos de un flujo de trabajo de ML, desde la preparación de los datos hasta la implementación del modelo. Con la información de seguimiento, puede reproducir los pasos del flujo de trabajo, hacer un seguimiento del linaje de modelo y del conjunto de datos y establecer normas de auditoría y gobernanza del modelo.

### Tarea 2.5.1: configurar la sesión y las variables

In [ ]:
#set-variables
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'

### Tarea 2.5.2: mostrar los valores que se utilizarán para construir el linaje del modelo

Las configuraciones incluyen las siguientes características:
- **query_string:** esta es la consulta de SageMaker Feature Store que se pasará al módulo MLLineageHelper.
- **model_ref:** este es el nombre del modelo que se evalúa.
- **processing_job:** este es el nombre del trabajo de procesamiento que generó el modelo.

In [ ]:
#print-values
print ('query_string:',query_string)

model_ref = sagemaker_client.list_models(SortBy = 'CreationTime', SortOrder = 'Descending')['Models'][0]['ModelName']
print ('model_ref:',model_ref)

processing_job = sagemaker_client.list_processing_jobs(SortBy = 'CreationTime', SortOrder = 'Descending', NameContains = 'ChurnModelProcess')['ProcessingJobSummaries'][0]['ProcessingJobName']
print ('processing_job:',processing_job)

processing_job_description = sagemaker_client.describe_processing_job(
    ProcessingJobName = processing_job
    )

### Tarea 2.5.3: describir el trabajo de procesamiento

In [ ]:
#describe-processing-job
processing_job_description

### Tarea 2.5.4: mostrar el nombre del trabajo de procesamiento que se usó para crear el modelo

In [ ]:
#print-training-job
training_job_name  =  sagemaker_client.list_training_jobs(SortBy = 'CreationTime', SortOrder = 'Descending')['TrainingJobSummaries'][0]['TrainingJobName']
print (training_job_name)

### Tarea 2.5.5: crear el linaje para el modelo

Si recibe el siguiente error, ejecute la celda otra vez.
- **ClientError: An error occurred (ThrottlingException) when calling the UpdateArtifact operation (reached max retries: 4): Rate exceeded** (ClientError: se ha producido un error [ThrottlingException] al llamar a la operación UpdateArtifact [reintentos máximos alcanzados: 4]: Tasa excedida**)

In [ ]:
#build-lineage
ml_lineage = MLLineageHelper()
lineage = ml_lineage.create_ml_lineage(training_job_name, model_name = model_ref,
                                       query = query_string, sagemaker_processing_job_description = processing_job_description,
                                       feature_group_names = [feature_group_name])

### Tarea 2.5.6: limitar el linaje para incluir solamente la prueba y el grupo de funciones actuales

Una canalización se puede ejecutar múltiples veces. Para asegurarse de obtener los detalles de la ejecución del trabajo de entrenamiento más reciente, filtre la llamada de linaje con el nombre de la prueba y del grupo de funciones actuales que usa la prueba. 

Después de ejecutar la celda, los pasos que usó para crear al modelo, el orden en el que se ejecutaron los pasos y los trabajos que contribuyeron con otros trabajos en la canalización se muestran en forma de tabla. Esta misma información también se escribe en un archivo llamado **lineage_FS.csv**. Puede descargar este archivo para guardar la salida y compartirla con otros miembros del equipo, como los auditores.

In [ ]:
#limit-lineage
trial_name = RunPipeline.describe()['PipelineExperimentConfig']['TrialName']
pat = str(trial_name)+'|'+'fg-FG'
df1 = lineage[lineage.apply(lambda x: any(x.str.contains(pat)),axis = 1)]
pd.set_option('display.max_colwidth', 120)
df1.to_csv('lineage_FS.csv') 
df1

### Tarea 2.5.7: generar una visualización del linaje del modelo

In [ ]:
#visualize-lineage
plt.figure(3, figsize = (20, 14))
graph = nx.DiGraph()
graph.add_edges_from([(each[0], each[2]) for each in df1.values])
fig, ax = plt.subplots()
nx.draw_networkx(
    graph,
    node_size = 300,
    node_color = "orange",
    alpha = 0.65,
    font_size = 8,
    pos = nx.spring_layout(graph)
)
ax.set_facecolor('deepskyblue')
ax.axis('off')
fig.set_facecolor('deepskyblue')
plt.show()

### Tarea 2.6: eliminar la canalización

Para borra la canalización, ejecute la siguiente celda:

In [ ]:
#delete-pipeline
response = sagemaker_client.delete_pipeline(PipelineName = 'ChurnModelSMPipeline')
print (response)

### Conclusión 

¡Felicitaciones! Usó SageMaker Pipelines para automatizar la creación y el registro de un modelo. Aprendió a profundizar en cada paso de la canalización para identificar los parámetros, archivos y registros asociados. Sabe cómo identificar los activos que la canalización utilizó para generar el modelo, cómo hallar el modelo en el registro de modelos y cómo hallar y ver los informes de explicabilidad y sesgos que puede generar la canalización.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.

### Sugerencias y respuestas para la tarea 2.3.1.1
Sugerencia general: el **Step type** (Tipo de paso) se encuentra en la pestaña **Information** (Información).

1. En el paso llamado **ChurnHyperParameterTuning**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** Tuning (Ajuste)</br>
    - ¿Cuál fue el **Overall Best Training Job** (Mejor trabajo de entrenamiento en general) que se generó en este paso? </br>
    **Sugerencia:** esta información se encuentra en la pestaña **Output** (Salida).</br>
    **Respuesta:** se genera el nombre del modelo, el cual será diferente para cada estudiante. El nombre debería ser algo similar a este ejemplo: 056vhzs2vkxc-ChurnHy-TCAtUr16oV-001-17d5bd01
1. En el paso **ChurnEvalBestModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?
    **Respuesta:** Processing (Procesamiento)</br>
    - ¿Cuál es el nombre del script de Python que se utiliza para evaluar el modelo superior que se identificó en el paso anterior?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Input** (Entrada).</br>
    **Respuesta:** evaluate.py</br>
    - ¿Dónde se ubica el archivo?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Input** (Entrada).</br>
    **Respuesta:** el archivo se encuentra en el bucket de S3. La ruta es similar a este ejemplo: s3://sagemaker-us-west-2-1234567890/input/code/evaluate.py</br>
    - ¿Dónde se escriben los resultados de este paso?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Output** (Salida).</br>
    **Respuesta:** los resultados de la evaluación se escribieron en un bucket de S3. La ruta del archivo debería ser similar al siguiente ejemplo: s3://sagemaker-us-west-2-1234567890/output/evaluation</br>
1. En el paso **CheckAUCScoreChurnEvaluation**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** Condition (Condición)</br>
    - ¿Cuál fue el **Evaluation outcome** (Resultado de la evaluación)?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Output** (Salida).</br>
    **Respuesta:** True (Verdadero)
1. En el paso **ChurnCreateModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** Model (Modelo)</br>
    - ¿Este trabajo generó algún registro?</br>
    **Respuesta:** no
1. En el paso **RegisterChurnModel**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** RegisterModel</br>
    - ¿Cuál es el valor de la métrica AUC?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Output** (Salida).</br>
    **Respuesta:** el valor variará, pero debería estar cerca de 0,98.
1. En el paso **ChurnTransform**, ubique los siguientes detalles:
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** Transform (Transformación)</br>
    - ¿Este trabajo generó registros?</br>
    **Respuesta:** sí</br>
    - ¿Qué archivos fueron las entradas para este trabajo?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Input** (Entrada). Es posible que deba deslizarse hasta la parte inferior del panel para hallar los nombres de los archivos.</br>
    **Respuesta:** model.tar.gz, sagemaker-xgboost:1.5-1-cpu-py3, batch.csv
1. En el paso **ChurnModelConfigFile**, ubique los siguientes detalles: 
    - ¿Qué ProcessingInstanceType (Tipo de instancia de procesamiento) se utilizó para ejecutar esta trabajo?</br>
    **Sugerencia:** esta información se encuentra en la pestaña **Input** (Entrada).</br>
    **Respuesta:** ml.m5.xlarge
    - ¿Cuál es el **Step Type** (Tipo de paso) de este paso?</br>
    **Respuesta:** Processing (Procesamiento)
1. En el **ClarifyProcessingStep**, ubique los siguientes detalles:
    - ¿Cuál fue el archivo de salida de este paso?
    **Sugerencia:** esta información se encuentra en la pestaña **Output** (Salida).</br>
    **Respuesta:** la salida eran datos sesgados.
    - ¿Dónde se escribió la salida?
    **Respuesta:** la salida se escribió en un bucket de S3. La ruta debería ser similar a este ejemplo: s3://sagemaker-us-west-2-1234567890/clarify-output/bias